In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv

# Path to the ChromeDriver executable
chromedriver_path = 'C:/Users/Amal Maatoug/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe' 


service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)


url = 'https://fr.shopping.rakuten.com/nav/Tel-PDA_Telephones-mobiles'

driver.get(url)
  
time.sleep(20)

data = []

# Extraire les données des produits de la page actuelle
def extract_data():
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    selector = '.mh-16.productList_childrenLayout_d73'
    containers = soup.select(selector)
    
    for product in containers:
        title_tag = product.find('p', class_='description_styleTitle_KPO lh-title normal mv0 b di')  
        neuf_price_tag = product.find('span', class_='f20 dib b')
        occasion_price_tag = product.find('span', class_='b dib f20')
        description_tag=product.find('ul',class_='description_bulletPoints_qgD')

        title = title_tag.text.strip() if title_tag else 'N/A'
        neuf_price = neuf_price_tag.text.strip() if neuf_price_tag else 'N/A'
        occasion_price = occasion_price_tag.text.strip() if occasion_price_tag else 'N/A'
        description=description_tag.text.strip() if description_tag else 'N/A'

        data.append({
            'title': title,
            'neuf_price': neuf_price,
            'occasion_price': occasion_price,
            'description':description
        })


extract_data()

# Fermer le navigateur
driver.quit()


In [4]:
# Le chemin du fichier CSV
csv_file_path = 'extracted_data.csv'

# Écrire les données dans un fichier CSV
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['title', 'neuf_price', 'occasion_price','description'])
    writer.writeheader()
    writer.writerows(data)

print(f"Les données ont été écrites dans {csv_file_path}")

Les données ont été écrites dans extracted_data.csv


In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup
import csv
import time

chromedriver_path = 'C:/Users/Amal Maatoug/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe' 

service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

url = 'https://fr.shopping.rakuten.com/nav/Tel-PDA_Telephones-mobiles'

driver.get(url)

data = []

# Fonction pour extraire les données de la page actuelle
def extract_data():
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    selector = '.mh-16.productList_childrenLayout_d73'
    containers = soup.select(selector)
    
    for product in containers:
        title_tag = product.find('p', class_='description_styleTitle_KPO lh-title normal mv0 b di')
        neuf_price_tag = product.find('span', class_='f20 dib b')
        occasion_price_tag = product.find('span', class_='b dib f20')
        description_tag=product.find('ul',class_='description_bulletPoints_qgD')


        title = title_tag.text.strip() if title_tag else 'N/A'
        neuf_price = neuf_price_tag.text.strip() if neuf_price_tag else 'N/A'
        occasion_price = occasion_price_tag.text.strip() if occasion_price_tag else 'N/A'
        description=description_tag.text.strip() if description_tag else 'N/A'


        data.append({
            'Article': title,
            'neuf_price': neuf_price,
            'occasion_price': occasion_price,
            'description':description
        })

# Extraire les données de la première page
extract_data()

# Boucler sur les pages suivantes jusqu'à la dernière
try:
    while True:
        # Trouver et cliquer sur le bouton "Suivant"
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//div[@data-qa="nextButton"]/a'))
        )
        next_button.click()
        
        # Attendre que la nouvelle page se charge
        time.sleep(30)  
        
        # Extraire les données de la page actuelle
        extract_data()

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Fermer le navigateur
    driver.quit()

    # le chemin du fichier CSV
    csv_file_path = 'extracted_data2.csv'

    # Écrire les données dans un fichier csv
    with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['Article', 'neuf_price', 'occasion_price','description'])
        writer.writeheader()
        writer.writerows(data)

    print(f"Les données ont été écrites dans {csv_file_path}")


An error occurred: Message: element click intercepted: Element <a class="flex outline-0 no-underline" rel="" href="/nav/Tel-PDA_Telephones-mobiles?pa=4">...</a> is not clickable at point (705, 823). Other element would receive the click: <div class="kml-modale image" style="display: block;">...</div>
  (Session info: chrome=126.0.6478.115)
Stacktrace:
	GetHandleVerifier [0x00007FF691A83E42+31618]
	(No symbol) [0x00007FF6919FB0A9]
	(No symbol) [0x00007FF6918B888A]
	(No symbol) [0x00007FF69191022E]
	(No symbol) [0x00007FF69190DBE2]
	(No symbol) [0x00007FF69190B07B]
	(No symbol) [0x00007FF69190A246]
	(No symbol) [0x00007FF6918FC281]
	(No symbol) [0x00007FF69192D10A]
	(No symbol) [0x00007FF6918FBBA6]
	(No symbol) [0x00007FF69192D320]
	(No symbol) [0x00007FF69194CA80]
	(No symbol) [0x00007FF69192CEB3]
	(No symbol) [0x00007FF6918FA46B]
	(No symbol) [0x00007FF6918FB001]
	GetHandleVerifier [0x00007FF691D8A01D+3202397]
	GetHandleVerifier [0x00007FF691DD6A3D+3516285]
	GetHandleVerifier [0x00007F